Olá, neste notebook, vamos aprender como podemos identificar o trajeto para um robô seguidor de linha, mesmo quando existe elementos como marcações ao lado ou efeitos de _glare_.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual, IntSlider, FloatSlider

image_path = "glare.jpeg"
image = cv2.imread(image_path)

threshed = None
eroded = None

plt.imshow(image)

Traceback (most recent call last):


Primeiro, já poderiamos utilizar _threshholds_ de cores para separar o fundo da linha branca. Use os arrastadores para ajustar os valores!

In [ ]:
def threshhold(threshA=0, threshB=255):
    global image
    global threshed
    a, b = threshA, threshB
    lower_bgr_values = np.array([a,  a,  a])
    upper_bgr_values = np.array([b, b, b])

    threshed = cv2.inRange(image, lower_bgr_values, upper_bgr_values)
    plt.imshow(threshed)

interact(threshhold, threshA=IntSlider(min=1, max=255, step=1, value=100),
                     threshB=IntSlider(min=0, max=255, step=1, value=255))


Traceback (most recent call last):


Aplicando a operação de erosão, podemos separar objetos que antes estavam conectados, e deformar objetos que não tinham forma bem definida, como o _glare_. Mude o tamanho do _kernel_ e veja o resultado!

In [ ]:
def erode(size=10):
    global threshed
    global eroded
    kernel = np.ones((size, size), np.uint8)
    eroded = cv2.erode(threshed, kernel)
    plt.imshow(eroded)

interact(erode, size=IntSlider(min=0, max=30, step=1, value=0))

Traceback (most recent call last):


Agora, na parte final, 

In [ ]:
MIN_AREA = 10000
MIN_AREA_TRACK = 30000
MAX_CONTOUR_VERTICES = 50

def getContour(eps=1.5, MIN_AREA_TRACK=30000, MAX_CONTOUR_VERTICES=50):
    """
    Return the centroid of the largest contour in
    the binary image 'mask' (the line)
    and draw all contours on 'out' image
    """

    global image
    global eroded
    out = image.copy()

    # get a list of contours
    contours, _ = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    mark = {}
    line = {}
    over = False
    tried_once = False

    possible_tracks = []
    for contour in contours:
        M = cv2.moments(contour)
        # Search more about Image Moments on Wikipedia 🙂 (it's the 'center')

        contour = cv2.approxPolyDP(contour, eps, True)
        contour_vertices = len(contour)


        if (M['m00'] < MIN_AREA):
            continue

        if (contour_vertices < MAX_CONTOUR_VERTICES) and (M['m00'] > MIN_AREA_TRACK):

            # Contour is part of the track
            line['x'] = int(M["m10"]/M["m00"])
            line['y'] = int(M["m01"]/M["m00"])

            possible_tracks.append(line)

            # plot the amount of vertices in light blue
            # cv2.drawContours(out, contour, -1, (255,255,0), 2)
            cv2.drawContours(out, [contour], -1, (255,255,0), 2)
            cv2.putText(out, f"Track", (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])-200),
                cv2.FONT_HERSHEY_PLAIN, 3, (100,100,255), 4)

        else:
            # plot the area in pink
            cv2.drawContours(out, contour, -1, (255,0,255), 4)
            cv2.putText(out, f"Ignored", (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])),
                cv2.FONT_HERSHEY_PLAIN, 3, (100,100,255), 4)

    for line in possible_tracks:
        cv2.circle(out, (line['x'], line['y']), 5, (0,255,0), 5)

    plt.imshow(out)

interact(getContour, eps=FloatSlider(min=0, max=15, step=0.1, value=0.5), MIN_AREA_TRACK=IntSlider(min=10000, max=50000, step=10000), MAX_CONTOUR_VERTICES=IntSlider(min=20, max=70, step=5))

Traceback (most recent call last):


Com isso, o robô pode calcular o centro deste contorno e descobrir onde deve ir!